# LightGBM + Dask

<table>
    <tr>
        <td>
            <img src="./_img/lightgbm.svg" width="300">
        </td>
        <td>
            <img src="./_img/dask-horizontal.svg" width="300">
        </td>
    </tr>
</table>

This notebook shows how to use `lightgbm.dask` to train a LightGBM model on data stored as a [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html) or [Dask Array](https://docs.dask.org/en/latest/array.html).

It uses `LocalCluster` to run on this machine only. If you want to try with a distributed cluster on AWS Fargate, see [the AWS notebook](./aws.ipynb).

In [ ]:
import dask.array as da

from dask.distributed import Client, LocalCluster, wait

from lightgbm.dask import DaskLGBMRegressor

Create a cluster with 3 workers. Since this is a `LocalCluster`, those workers are just 3 local processes.

In [ ]:
n_workers = 3
cluster = LocalCluster()
client = Client(cluster)
client.wait_for_workers(n_workers)

print(f"View the dashboard: {cluster.dashboard_link}")

Click the link above to view a diagnostic dashboard while you run the training code below.

In [ ]:
num_rows = 1e6
num_features = 1e2
num_partitions = 10
rows_per_chunk = num_rows / num_partitions

data = da.random.random((num_rows, num_features), (rows_per_chunk, num_features))

labels = da.random.random((num_rows, 1), (rows_per_chunk, 1))

Right now, the Dask Arrays `data` and `labels` are lazy. Before training, you can force the cluster to compute them by running `.persist()` and then wait for that computation to finish by `wait()`-ing on them.

In [ ]:
data = data.persist()
labels = labels.persist()
_ = wait(data)
_ = wait(labels)

With the data set up on the workers, train a model. `lightgbm.dask.DaskLGBMRegressor` has an interface that tries to stay as close as possible to the non-Dask scikit-learn interface to LightGBM (`lightgbm.sklearn.LGBMRegressor`).

In [ ]:
dask_reg = DaskLGBMRegressor(
    silent=False,
    max_depth=5,
    random_state=708,
    objective="regression_l2",
    learning_rate=0.1,
    tree_learner="data",
    n_estimators=10,
    min_child_samples=1,
    n_jobs=-1,
)

dask_reg.fit(
    client=client,
    X=data,
    y=labels,
)

The model produced by this training run is an instance of `DaskLGBMRegressor`. To get a regular non-Dask model (which can be pickled and saved), run `.to_local()`.

In [ ]:
local_model = dask_reg.to_local()
type(local_model)

You can visualize this model by looking at a data frame representation of it.

In [ ]:
local_model.booster_.trees_to_dataframe()